In [ ]:
# If you use Autograd, please run this cell instead of JAX cell below
import autograd.numpy as np
from autograd import grad

In [ ]:
# Run this cell if you use JAX
from jax import numpy as np
from jax import grad

In [ ]:
import numpy as tnp
from sklearn.datasets import make_moons

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from scipy.optimize import minimize

In [ ]:
x, y = make_moons(2000, noise=0.1, shuffle=False)

x2,y2 = make_moons(2000, noise=0.1, shuffle=False)
x2 = x2[y2==0]
y2 = y2[y2==0]
y2 = y2+2
x2 = x2*2.5
x2[:,0] = x2[:,0]+0.5


x = tnp.concatenate([x,x2], axis=0)

y = tnp.concatenate([y,y2])

In [ ]:
x.shape

In [ ]:
colors = {0: 'red', 1: 'green', 2: 'blue', 3: 'gray'}
cmaps = ['Reds', 'Greens', 'Blues', 'Greys']
clabels = [colors[l] for l in y]

In [ ]:
f = plt.figure(figsize=(3,3), dpi=300)
plt.scatter(x[:,0], x[:,1], s=1, c=clabels)
_ = plt.axis('equal')

In [ ]:
def one_hot(arr):
    classes = np.unique(y)
    num_classes = len(classes)
    return classes,tnp.squeeze(tnp.eye(num_classes)[arr.reshape(-1)])

In [ ]:
classes,y_oh = one_hot(y)

In [ ]:
y_oh

In [ ]:
def softmax(z):
    # z ~ N x C3
    
    # YOUR CODE HERE
    # you need to implement numerically stable Softmax function of z
    
    softmax_values = np.zeros_like(z)
    
    return softmax_values

In [ ]:
def relu(z):
    # YOUR CODE HERE
    # you need to implement ReLU function of z
    # the resulting matrix should be assigned to the "result" variable
    
    result = np.zeros_like(z)
    
    return result

In [ ]:
def compute_probabilities(X, theta):
    # you need to implement a neural network that computes the estimates of probabilities
    # for each object of X for each class

    # note that the size of p should be NxK
    # where
    # N is the number of objects in X
    # K is the number of classes of the problem
    
    # (1) you need to get chunks of theta values out of theta of reasonable lengths
    #     and reshape them into the correct theta matrices for each ANN layer
    # (2) you need to implement layers themselves,
    #     which includes matrix multiplication and the application of a non-linearity function
    #     (sigmoid, ReLU, softmax, etc.)
    #     make sure you did not forget biases for each layer
    
    # p ~ N x C3
    # C3 = 3
    
    f = X.shape[1]
    # Here is the configuration of the network
    f1 = 6
    f2 = 4
    f3 = 3
    
    p = np.zeros((X.shape[0], f3))
    
    # YOUR CODE HERE
    
    return p

In [ ]:
def multinomial_cross_entropy(p_pred, y_true):
    CE = 0
    return CE

In [ ]:
def multinomial_cross_entropy_loss(X, y, theta, reg_alpha = 1.0e-3):
    # YOUR CODE HERE
    
    p = np.zeros((X.shape[0], 3))
    loss = 0
    
    # YOUR ADDITIONAL CODE HERE
    # you may implement the regularization L2 penalty term
    # that helps to limit the values of parameters theta of the model
    
    loss = loss + 0
    
    return loss

In [ ]:
# you need to generate random theta values according to:
#          - the number of features of input data X
#          - the number of nodes in each layer, and the bias term
#          - the number of output nodes (the number of classes K)
# Then you need to flatten these matrices and and concatenate them into one vector
# (due to the specifics of the function "minimize" of scipy package)
#
# assign the vector of theta values to the vatiable theta_start

f = x.shape[1]
f1 = 6
f2 = 4
f3 = 3

theta_start = None
# YOUR CODE HERE

In [ ]:
theta_length

In [ ]:
theta_start

### Here we are starting to optimize the model we have just created

In [ ]:
loss_fn = multinomial_cross_entropy_loss

In [ ]:
grad_fn = grad(loss_fn, argnums=2)

In [ ]:
curr_loss = loss_fn(x,y_oh,theta_start)

In [ ]:
curr_loss

In [ ]:
curr_loss_grad = grad_fn(x,y_oh,theta_start)
curr_loss_grad

In [ ]:
# this is just a callback that runs each iteration of the minimization loop
# we are just logging the loss history here

def minimization_callback(loss_history, curr_loss_value):
    loss_history.append(curr_loss_value)
    print('loss_value: %f' % curr_loss_value)

In [ ]:
# actual minimization of the loss function of out model

loss_history = []
optimization_result = minimize(lambda t: float(loss_fn(x, y_oh, t)),
                               theta_start,
                               jac = lambda t: np.array(grad_fn(x, y_oh, t)).flatten(),
                               callback = lambda t: minimization_callback(loss_history, float(loss_fn(x,y_oh,t))))

In [ ]:
# let us plot the loss function evolution

plt.plot(loss_history)
plt.yscale('log')

In [ ]:
# here we will get the theta values from the optimization result

theta_result = optimization_result.x

In [ ]:
theta_result

In [ ]:
curr_loss_grad = grad_fn(x,y_oh,theta_result)
curr_loss_grad

In [ ]:
# computing probabilities using the optimized ANN weights and
# its architecture described in the function compute_probabilities()

pred_proba = compute_probabilities(x, theta_result)
print(pred_proba.shape)

In [ ]:
# transforming the probabilities into the class labels

y_pred = classes[np.argmax(pred_proba, axis=1)]

In [ ]:
y_pred

In [ ]:
# calculating the accuraacy score

np.mean(y_pred == y)

### Let us plot the resulting probabilities for the classes

In [ ]:
nbins = 400
xmesh, ymesh = tnp.mgrid[-4:4:nbins*1j, -4:4:nbins*1j]

In [ ]:
x_test_mesh = np.concatenate([xmesh.ravel()[:,np.newaxis], ymesh.ravel()[:,np.newaxis]], axis=-1)
x_test_mesh.shape

In [ ]:
probas_mesh = compute_probabilities(x_test_mesh, theta_result)

In [ ]:
preds_mesh = classes[np.argmax(probas_mesh, axis=1)]

In [ ]:
probas_mesh = probas_mesh.reshape(list(xmesh.shape) + [3])

In [ ]:
preds_mesh = preds_mesh.reshape(xmesh.shape)

In [ ]:
_ = plt.figure(figsize=(6,5), dpi=300)
plt.scatter(x[:,0], x[:,1], s=1, c=clabels)
for class_label,proba_mesh in zip([0,1,2], [probas_mesh[:,:,0], probas_mesh[:,:,1], probas_mesh[:,:,2]]):
    pm = tnp.ma.array(proba_mesh)
    pm.mask = (preds_mesh != class_label)
    _ = plt.pcolormesh(xmesh, ymesh, pm, cmap=cmaps[class_label], alpha=0.5)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(x, y)

In [ ]:
LR_preds = LR.predict(x)

In [ ]:
# calculating the accuraacy score

np.mean(LR_preds == y)

In [ ]:
LR_probas_mesh = LR.predict_proba(x_test_mesh)

In [ ]:
LR_probas_mesh.shape

In [ ]:
LR_preds_mesh = classes[np.argmax(LR_probas_mesh, axis=1)]

In [ ]:
LR_probas_mesh = LR_probas_mesh.reshape(list(xmesh.shape) + [3])

In [ ]:
LR_preds_mesh = LR_preds_mesh.reshape(xmesh.shape)

In [ ]:
_ = plt.figure(figsize=(6,5), dpi=300)
plt.scatter(x[:,0], x[:,1], s=1, c=clabels)
for class_label,proba_mesh in zip([0,1,2], [LR_probas_mesh[:,:,0], LR_probas_mesh[:,:,1], LR_probas_mesh[:,:,2]]):
    pm = tnp.ma.array(proba_mesh)
    pm.mask = (preds_mesh != class_label)
    _ = plt.pcolormesh(xmesh, ymesh, pm, cmap=cmaps[class_label], alpha=0.5)